In [ ]:
import sys
from google.colab import drive

drive.mount('/content/drive')
sys.path.append(r"/content/drive/MyDrive/")

Mounted at /content/drive


In [1]:
import sys, os, json
import numpy as np
import pandas as pd
import seaborn as sns
import argparse, progressbar
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
import matplotlib.axes._axes as axes

sns.set()

In [2]:
# import the necessary packages
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from loader_util.preprocessing import ImageToArrayPreprocessor, AspectAwarePreprocessor, SimplePreProcessor, MeanPreprocessor
from loader_util.io import HDF5DatasetGenerator
from loader_util.datasets import SimpleDatasetLoader
from loader_util.nn.conv import FCHeadNet, ResNet
from loader_util.callbacks import EpochCheckpoint, TrainingMonitor
##
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Conv2D, Activation, Flatten, Dense, BatchNormalization, MaxPooling2D, AveragePooling2D, Dropout
from tensorflow.keras.layers import Input, concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.applications import VGG16
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model, load_model

In [3]:
args = {
    "checkpoints": f"/content/drive/MyDrive/ImageDatasets/resnet/output/checkpoints",
    "model_path": f"",
    "start_epoch": 0
}

In [4]:
print(f"[INFO] loading cifar data......")
(trainx, trainy), (testx, testy) = cifar10.load_data()
testx = testx.astype(np.float)
trainx = trainx.astype(np.float)

[INFO] loading cifar data......


c:\users\mhasa\pycharmprojects\dl4cv\venv\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\mhasa\pycharmprojects\dl4cv\venv\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [5]:
# apply mean sub
mean = np.mean(trainx, axis=0)
trainx -= mean
testx -= mean

# encode labels
le = LabelBinarizer()
trainy_encoded = le.fit_transform(trainy)
testy_encoded = le.transform(testy)

In [6]:
# construct image augmentor
aug = ImageDataGenerator(width_shift_range=0.1,
                         height_shift_range=0.1,
                         horizontal_flip=True,
                         fill_mode='nearest')

In [7]:
if not args["model_path"]:
    print(f"[INFO] compiling model......")
    opt = SGD(learning_rate=1e-1)

    model = ResNet.build(width=32, height=32, depth=3, classes=10,
                         stages=(9, 9, 9),
                         filters=(64, 64, 128, 256),
                         reg=0.0005)
    model.compile(loss="categorical_crossentropy",
                  optimizer=opt,
                  metrics=["accuracy"])
else:
    print(f"[INFO] loading {args['model_path']}......")
    model = load_model(args['model_path'])  # type: Model

    # update learning rate
    print(f"[INFO] old learning rate: {K.get_value(model.optimizer.lr)}......")
    K.set_value(model.optimizer.lr, 1e-5)
    print(f"[INFO] new learning rate: {K.get_value(model.optimizer.lr)}......")

[INFO] compiling model......


In [8]:
# print model summary
model.summary()

Model: "resnet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 3)    12          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 64)   1728        batch_normalization[0][0]        
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 64)   256         conv2d[0][0]                     
_____________________________________________________________________________________________

In [9]:
# construct callbacks
callbacks = [
    EpochCheckpoint(outputPath=args["checkpoints"], every=5, startAt=args["start_epoch"]),
    TrainingMonitor(figPath="/content/drive/MyDrive/ImageDatasets/resnet/output/checkpoints/resnet_cifar10.png", jsonPath="/content/drive/MyDrive/ImageDatasets/resnet/output/checkpoints/resnet_cifar10.json", startAt=args["start_epoch"])
]

In [10]:
# train the net
batch_size = 32
print(f"[INFO] training the net......")
H = model.fit_generator(aug.flow(trainx, trainy_encoded, batch_size=batch_size),
                        validation_data=(testx, testy_encoded),
                        steps_per_epoch=len(trainx) // batch_size,
                        epochs=100,
                        callbacks=callbacks,
                        verbose=1)

[INFO] training the net......
Epoch 1/100
  58/1562 [>.............................] - ETA: 1:20:09 - loss: 2.6028 - accuracy: 0.2117

KeyboardInterrupt: 

In [ ]:
import time
while True:
  time.sleep(10)